In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pylsd.lsd import lsd
%matplotlib inline

from modules import preprocessing

In [2]:
img_path = "/Users/Chihiro/Personal/10_Projects/GPC/get_plant_plate_txt/image/20200411_1.jpg"
output_dir = "/Users/Chihiro/Desktop/"

In [3]:
def get_lines(base_img, img):
    
    return_img = np.full_like(img, 255, dtype=np.uint8)
    #直線検出（ハフ変換)
#     lines = cv2.HoughLines(img, 
#                         rho=1, 
#                         theta=np.pi/360, 
#                         threshold=50)

     # 直線検出（確率的ハフ変換）
    lines = cv2.HoughLinesP(img, 
                                                rho=1, 
                                                theta=np.pi/360, 
                                                threshold=50, 
                                                minLineLength=500, 
                                                maxLineGap=3)

#     #直線検出（LSD）
#     lines = lsd(img)
    
    print(len(lines))

    
    for line in lines:
        x1, y1, x2, y2 = line[0]
        
        

        # 赤線を引く
        line_img = cv2.line(return_img, (x1,y1), (x2,y2), (0,0,255), 10)

    return return_img

In [4]:
def binalize(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     img_gray = cv2.GaussianBlur(img_gray,(5,5),0)
    
    
    #  二値化 
#     low_thr = 200
#     hight_thr = 255
#     retval, img_bin = cv2.threshold(img_gray,low_thr,hight_thr,cv2.THRESH_BINARY)
    
    # 大津の二値化 
    low_thr = 0
    hight_thr = 255
    retval, img_bin = cv2.threshold(img_gray, low_thr, hight_thr, cv2.THRESH_OTSU)

#     適応的閾値処理による二値化
#     img_bin = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 39,2)
    
    return img_bin

In [43]:
def get_contour(org_img, img):
    
    contours_img = org_img
    
    #   領域の抽出
    img, contours, hierarchy = cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)



     # 面積の表示
    contour_areas = {}
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        contour_areas[i] = area
        
    
    
#     面積が最大のindexを取得（面積が大きすぎるものは省いた方が良い）
#     max_area = sorted(contour_areas.values(),reverse=True)[0]
    max_area = max(contour_areas.values())
    max_area_idx = [i for i, v in contour_areas.items() if v == max_area][0]
    
    max_contour = contours[max_area_idx]
    arc_len = cv2.arcLength(max_contour, True)
        
    # 輪郭を近似
    approx_contour = cv2.approxPolyDP(max_contour, epsilon=0.05 * arc_len, closed=True)
    
    cv2.drawContours(contours_img, approx_contour, -1, (0,255,0), 30)
    cv2.imwrite(f"{output_dir}approx_contour.png",contours_img)

    return approx_contour

In [6]:
def noise_reduction(img):
#     kernel = np.ones((10,10)) # カーネルサイズの調整が必要そう
#     img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    img = cv2.medianBlur(img,15)
    return img

In [25]:
img = cv2.imread(img_path)
masked_img = preprocessing.mask_img(img)

In [34]:
img = noise_reduction(masked_img)

In [35]:
org_img = cv2.imread(img_path)
bin_img = binalize(img)
edge_img = cv2.Canny(bin_img,10,255,apertureSize = 3)

In [36]:
# 画像の膨張・縮小
kernel = np.ones((5,5),np.uint8)

dilation_img = cv2.dilate(edge_img,kernel,iterations = 10)
erode_img = cv2.erode(dilation_img,kernel,iterations = 3)

In [46]:
# プレート四隅の座標取得
contour = get_contour(org_img, erode_img)

In [97]:
# 歪み補正
def get_plate_img(contour) :
    approx=contours_points.tolist()

    left = sorted(approx,key=lambda x:x[0]) [:2]
    right = sorted(approx,key=lambda x:x[0]) [2:]

    left_down= sorted(left,key=lambda x:x[0][1]) [0]
    left_up= sorted(left,key=lambda x:x[0][1]) [1]
    right_down= sorted(right,key=lambda x:x[0][1]) [0]
    right_up= sorted(right,key=lambda x:x[0][1]) [1]



    perspective_base = np.float32([left_down,right_down,right_up,left_up])
    perspective = np.float32([[0, 0],[700, 0],[700, 500],[0, 500]])


    psp_matrix = cv2.getPerspectiveTransform(perspective_base, perspective)
    img_psp = cv2.warpPerspective(org_img, psp_matrix,(700,500))


return img_psp

True

In [96]:
cv2.imwrite(f"{output_dir}psp_erode_img.png",psp_erode_img)

True